<a href="https://colab.research.google.com/github/akinuzelgencer/Courses-/blob/master/Copy_of_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/emres/turkish-deasciifier.git
!pip install TurkishStemmer

  Cloning https://github.com/emres/turkish-deasciifier.git to /tmp/pip-req-build-6ob1denq
  Running command git clone -q https://github.com/emres/turkish-deasciifier.git /tmp/pip-req-build-6ob1denq
  Created wheel for Turkish-Deasciifier: filename=Turkish_Deasciifier-0.1-cp36-none-any.whl size=90362 sha256=c37d6ef939ae1f42c57ed3692e9b0cf82bb12d9c311eadea76c0c12c40fef3d4
  Stored in directory: /tmp/pip-ephem-wheel-cache-pgq1fihp/wheels/7e/6d/9a/5fb9e7c4f793ddb81477d5edae4d448b7ee1717a05b78cc493
Successfully built Turkish-Deasciifier


In [0]:
# Kullanacağımız kütüphaneler
import re
from turkish.deasciifier import Deasciifier
from TurkishStemmer import TurkishStemmer

## Format sorunları
Unicode, dijital ve geleneksel medyada her bir karakter ve sembolleri benzersiz bir rakam yardımıyla oluşturmak için geliştirilen bir metin standartıdır. Unicode’un geliştirilmesinin asıl amacı ASCII (American Standart Code for Information Interchange) karakter kodlamasının daha gelişmiş ve stratejik bir sürümünün oluşturulabilmesidir.

- ASCII yalnızca Latin alfabesi için kullanılabilir, unicode evrenseldir
- Python 'string' verisi unicode formatındadır.

*çiçek*   ->  *Ã§iÃ§ek* 

** Deasciifier **
- ASCII formatındaki metinleri unicode formatına çevirmek için
- İngilizce klavyeyle yazılmış metinleri düzeltmek / hatalı karakterleri Türkçe karakterlerle değiştirmek için

*Dogal Dil Isleme* -> *Doğal Dil İşleme*

In [0]:
def deasciify(text):
    deasciifier = Deasciifier(text)
    text = deasciifier.convert_to_turkish()
    return text

In [4]:
text = 'Dogal Dil Isleme'
deasciify(text)

'Doğal Dil İşleme'

## Metin Temizleme
İşlenecek metinler çoğunlukla 'pis' veridir.
- Sistematik hatalar (cümle başı ve sonlarındaki gereksiz boşluklar, vs.)
- Yazım hataları (gramer hataları, gereksiz boşluk kullanımı, vs.) -> ('Oluur')
- Birden fazla noktalama işareti
- Büyük/küçük harflerin yanlış kullanımı -> 'OKul'
- Rakam kullanımı

Başka problemler:
- Emoji kullanımı
- Yabancı sözcük kullanımı

Emoji ve yabancı sözcük kullanımı duruma bağlı olarak tolere edilebilir, hatta yararlı olabilir.

*Ne biliyim bende olur dedim :/:/* -> *Ne bileyim ben de olur dedim* 

In [5]:
# Örnek cümle
text = '''  Ben;de    dedim  ki
olabilir1.
'''
text

'  Ben;de    dedim  ki\nolabilir1.\n'

In [6]:
# Cümlenin başındaki ve sonundaki gereksiz boşlukları çıkar
text = text.strip()
text

'Ben;de    dedim  ki\nolabilir1.'

In [7]:
# Bütün karakterleri küçük harfe dönüştür
text = text.lower()
text

'ben;de    dedim  ki\nolabilir1.'

In [8]:
# Rakamları sil
text = re.sub(r'\d+', '', text)
text

'ben;de    dedim  ki\nolabilir.'

In [9]:
# Noktalama işaretleriyle harfler arasına boşluk ekle
text = re.sub( r'([a-zA-Z])([:;,.!?])', r'\1 \2', text)
text = re.sub( r'([:;,.!?])([a-zA-Z])', r'\1 \2', text)
text

'ben ; de    dedim  ki\nolabilir .'

In [10]:
# Gereksiz tüm boşlukları sil
text = re.sub(r"\s+", ' ', text)
text

'ben ; de dedim ki olabilir .'

### Metin Temizleme: Özet
Yukarıdaki işlemleri tek bir fonksiyon altında toplayabiliriz.

In [0]:
def clean_text(text):
    text = text.strip()
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    
    text = re.sub( r'([a-zA-Z])([:;,.!?])', r'\1 \2', text)
    text = re.sub( r'([:;,.!?])([a-zA-Z])', r'\1 \2', text)
    
    text = re.sub(r"\s+", ' ', text)
    return text

## Metin Ön-İşleme
Dil modellerini eğitirken performansı arttırmak ve metinleri standart hale getirmek için uygulayacabileceğimiz başka adımlar da var:

- Noktalama işaretlerinin tamamının veya bir kısmının kaldırılması
- Çok sık kullanılan 'dolgu' kelimelerin (stop words) kaldırılması
- Kısaltma kelimeleri (ünvanlar, vs.) tam haline getirmek
- Kelime eklerini çıkarmak ('kitabı', 'kitaplarım' -> 'kitap')
- Kelimeleri kök haline getirmek ('güneş' -> 'gün')

Hangi ön-işleme adımlarını kullanacağımız eğittimiz dil modelinin özelliklerine ve kullanım amacımıza bağlı. Örneğin Word2Vec gibi bir kelime temsil modelinde noktalama işaretleri ve rakamları tutmamızın hem bir amacı olmaz hem de model performansını düşürür.

### Dolgu Kelimelerin Çıkarılması

In [12]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: ignored

In [0]:
with open('tr-stopwords', 'r') as f:
    stopwords = f.read().splitlines()

stopwords[:5]

['a', 'acaba', 'altı', 'altmış', 'ama']

In [0]:
def remove_stopwords(text, stopwords):
    text = ' '.join([word for word in text.split() if word not in stopwords])
    return text

text = 'o filmi izlemedim ama çok övüyorlar'
remove_stopwords(text, stopwords)

'filmi izlemedim övüyorlar'

### Kelime Kökü Bulma

In [0]:
stemmer = TurkishStemmer()
stemmer.stem("okuldakilerden")

'okul'